In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, classification_report
from keras.models import Sequential, Model 
from keras.layers import Dense, Input, concatenate, Activation, Dropout
from keras.optimizers import Adam
from tensorflow import set_random_seed
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from micromlgen import port

Using TensorFlow backend.
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\luigi.borriello2\.conda

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\luigi.borriello2\.conda\envs

In [2]:
RANDOM_SEED = 42

In [3]:
np.random.seed(RANDOM_SEED)
set_random_seed(RANDOM_SEED)

In [4]:
with open('data/X.pkl', 'rb') as f:
    X = pickle.load(f)

with open('data/y.pkl', 'rb') as f:
    y = pickle.load(f)

In [5]:
classes = np.unique(y).tolist()
for i in range(len(classes)):
    y = np.where(y==classes[i], i, y) 

Split in train and test sets

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review 

In [7]:
#y_train = pd.get_dummies(y_train).values
#y_test = pd.get_dummies(y_test).values

In [8]:
y_train

array([5, 3, 3, 1, 3, 2, 1, 3, 0, 0, 5, 5, 4, 0, 1, 5, 4, 1, 1, 4, 3, 0,
       3, 2, 5, 0, 4, 1, 3, 4, 5, 5, 1, 1, 1, 2, 3, 4, 3, 3, 1, 2, 1, 4,
       1, 0, 2, 3, 5, 2, 0, 2, 3, 2, 4, 5, 1, 5, 1, 2, 2, 4, 2, 1, 4, 3,
       0, 1, 1, 1, 2, 3, 5, 4, 1, 1, 0, 2, 3, 5, 2, 4, 0, 2, 3, 0, 4, 1,
       0, 3, 5, 0, 2, 2, 5, 4, 2, 5, 3, 3, 0, 5, 4, 4, 3, 4, 0, 5, 3, 4,
       2, 0, 1, 0, 0, 4, 4, 4, 3, 2, 4, 3, 4, 5, 1, 1, 5, 2, 5, 1, 5, 4,
       1, 1, 2, 3, 4, 5, 0, 2, 1, 3, 5, 4, 0, 0, 1, 1, 0, 4, 2, 0, 5, 2,
       0, 3, 5, 0, 1, 0, 2, 4, 0, 0, 2, 4, 1, 0, 4, 2, 2, 3, 2, 3, 2, 4,
       4, 2, 0, 3, 0, 5, 0, 4, 0, 5, 3, 1, 0, 2, 4, 4, 4, 3, 0, 2, 0, 0,
       2, 3, 5, 0, 0, 1, 2, 2, 5, 4, 1, 1, 3, 4, 5, 3, 2, 1, 5, 3, 2, 4,
       1, 4, 3, 0, 3, 3, 4, 0, 4, 1, 1, 0, 2, 1, 5, 0, 0, 3, 0, 2, 4, 4,
       3, 4, 4, 5, 5, 3, 4, 5, 5, 0, 4, 2, 1, 2, 5, 4, 4, 0, 5, 1, 1, 3,
       2, 5, 4, 4, 2, 1, 3, 1, 5, 0, 2, 5, 4, 4, 3, 4, 0, 3, 2, 1, 3, 4,
       1, 5, 1, 3, 0, 2, 3, 5, 4, 3, 2, 5, 4, 1, 0,

In [9]:
X_train.shape[1]

32

## Models Evaluation

### RF

In [10]:
model = RandomForestClassifier(n_estimators=500, random_state=42)
#model = MLPClassifier(random_state=42)

In [11]:
model.fit(X_train, y_train)

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision.

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [12]:
y_pred = model.predict(X_test)

C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [13]:
f1_score(y_test, y_pred, average="weighted")

0.8570874625393995

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.84      0.86        19
           1       0.86      0.90      0.88        20
           2       1.00      1.00      1.00        20
           3       0.70      0.70      0.70        20
           4       0.70      0.70      0.70        20
           5       1.00      1.00      1.00        20

    accuracy                           0.86       119
   macro avg       0.86      0.86      0.86       119
weighted avg       0.86      0.86      0.86       119



In [18]:
print(confusion_matrix(y_test, y_pred))

[[16  3  0  0  0  0]
 [ 2 18  0  0  0  0]
 [ 0  0 20  0  0  0]
 [ 0  0  0 14  6  0]
 [ 0  0  0  6 14  0]
 [ 0  0  0  0  0 20]]


In [19]:
with open('rf_model.h', 'w') as f:  # change path if needed
    f.write(port(model, optimize=False))

In [20]:
with open('models/rf_model.pkl', 'wb') as f:
    pickle.dump(model, f)


### MLP

In [21]:
# design network

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=32))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [22]:
opt = Adam(lr=1e-5, decay=1e-5)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
# fit network
history = model.fit(x=X_train, y=y_train, epochs=100, batch_size=16, validation_split=0.25, verbose=1, shuffle=True)

Train on 356 samples, validate on 119 samples
Epoch 1/100
356/356 [==============================] - 0s 687us/step - loss: -23.7327 - acc: 0.1713 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 2/100
356/356 [==============================] - 0s 84us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 3/100
356/356 [==============================] - 0s 15us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 4/100
356/356 [==============================] - 0s 72us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 5/100
356/356 [==============================] - 0s 79us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 6/100
356/356 [==============================] - 0s 75us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 7/100
356/356 [==============================] - 0s 60us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val

356/356 [==============================] - 0s 88us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 60/100
356/356 [==============================] - 0s 53us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 61/100
356/356 [==============================] - 0s 82us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 62/100
356/356 [==============================] - 0s 48us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 63/100
356/356 [==============================] - 0s 46us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 64/100
356/356 [==============================] - 0s 61us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 65/100
356/356 [==============================] - 0s 58us/step - loss: -23.6897 - acc: 0.1685 - val_loss: -24.2485 - val_acc: 0.1597
Epoch 66/100
356/356 [==================

In [24]:
model.save('models/rnn-cnn')

ImportError: `save_model` requires h5py.

In [ ]:
with open('models/rnn-cnn-history.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')
pyplot.legend()
pyplot.show()

In [187]:
# make a prediction
y_pred = model.predict(X_test)

In [188]:
np.argmax(y_pred, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [101]:
f1_score(y_test, np.argmax(y_pred, axis=1), average="weighted")

AxisError: axis 1 is out of bounds for array of dimension 1

In [81]:
print(classification_report(y_test, np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        19
           1       0.00      0.00      0.00        20
           2       0.00      0.00      0.00        20
           3       0.17      1.00      0.29        20
           4       0.00      0.00      0.00        20
           5       0.00      0.00      0.00        20

    accuracy                           0.17       119
   macro avg       0.03      0.17      0.05       119
weighted avg       0.03      0.17      0.05       119



c:\program files\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [84]:
print(confusion_matrix(y_test, y_pred.argmax(axis=1)))

[[ 0  0  0 19  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]
 [ 0  0  0 20  0  0]]
